## 2. Clasificación con SVM

Como se ha mencionado en el preprocesamiento, en el problema que aquí se estudia, cada individuo puede pertenecer a una clase, las dos o ninguna, por lo que se está tratando de un problema multietiqueta. Para usar una máquina de soporte vectorial para clasificar, te

In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.svm import LinearSVC, SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler



In [3]:
X_train = pd.read_csv("preprocessed_training_dataset.csv")
y_train = pd.read_csv("training_set_labels.csv")

# Cargar el dataset de prueba
X_test = pd.read_csv("preprocessed_test_dataset.csv")

In [5]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape)

Train set: (26707, 96) (26707, 3)
Test set: (26708, 96)


In [7]:
# Eliminar la columna de índice (asumiendo que es la primera columna)
X_train = X_train.iloc[:, 1:]
y_train = y_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

In [19]:
X_train.head()

,h1n1_concern,h1n1_knowledge,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
0,1.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.0,5.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,1.0,3.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,1.0,3.0,3.0,5.0,5.0,4.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,1.0,3.0,3.0,2.0,3.0,1.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Antes de proceder, es necesario escalar los valores, sobre todo para que las variables de mayor rango (pertenencientes a las preguntas de opinión mayoritariamente) no influyan en el cálculo de las distancias con SVM. Dentro de las posibilidades de escalado, se puede optar por:
- Normalización zero-mean: Escala los valores en un rango $[0,1]$ en función de la media y la varianza de la variable. Esta normalización se usa cuando las distribuciones de las variables siguen aproximadamente una distribución normal.
- Normalización MinMaxScaling: Escala los valores en un rango $[0,1]$ en función de los valores máximos y mínimos de cada variable. Esta técnica de escalado conserva las características de los datos originales tras la transformación. 

Dado que no se conocen las distribuciones originales de los datos, se va a optar por el uso de escalado MinMax

In [9]:
# Escalar características
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[0.66666667, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 0.5       , 0.75      , ..., 1.        , 0.        ,
        0.        ],
       [0.66666667, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.5       , 0.75      , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.5       , 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [0.66666667, 0.5       , 1.        , ..., 0.        , 1.        ,
        0.        ]])

In [11]:
# Asegurarse de que todos los valores sean numéricos
#X_train = X_train.astype(float)
#X_test = X_test.astype(float)
y_train = y_train.astype(int)

In [15]:
from sklearn.model_selection import train_test_split

X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42
)

Una vez tenemos todos los datos en el mismo formato, podemos pasar a implementar SVM. Este algoritmo trata de sobredimensionar los datos para poder generar un hiperplano de separación entre dos clases, por lo que, de manera nativa no es capaz de realizar una clasificación multietiqueta. Para solventar este problema se usa un pequeño 'truco' a la hora de implementar el algoritmo, que es la descomposición binaria de la respuesta obtenida, de esta manera SVM puede abordar los problemas multietiqueta como problemas independientes:

- Si se entrena un clasificador para cada par de valores posibles se está usando una estrategia OVO (One-vs-One)
- Si el clasificador es capaz de distinguir en cada caso su etiqueta frente a las demás se usa una estrategia OVR (One-vs-Rest)

Otra opción para la clasificación multietiqueta pasa por el uso de la función 'MultiOutputClassifier' implementada en 'sklearn', capaz de tratar cada etiqueta como un problema binario independiente, entrenando un clasificador para cada etiqueta sin comparar con las demás:

In [42]:
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)

# Paso 2: Configurar MultiOutputClassifier para manejar multietiqueta
multi_label_svm = MultiOutputClassifier(base_svm)

# Paso 3: Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'estimator__C': [0.1, 1],       # Regularización
    'estimator__gamma': ['scale', 'auto'],   # Parámetro del kernel RBF
    'estimator__kernel': ['rbf', 'poly'],    # Tipos de kernel
    'estimator__max_iter': [10000, 50000]
}

# Paso 4: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=multi_label_svm,
    param_grid=param_grid,
    cv=5,  # Validación cruzada de 5 pliegues
    scoring='accuracy',  # Métrica para evaluar el modelo
    verbose=3,  # Para ver el progreso
    n_jobs=-1   # Usar todos los núcleos disponibles
)

# Paso 5: Entrenar con los datos de entrenamiento
grid_search.fit(X_train_split, y_train_split)

# Paso 6: Resultados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report

y_valid_pred = best_model.predict(X_valid_split)
print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Mejores parámetros encontrados:  {'estimator__C': 1, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': 50000}
Accuracy en validación: 0.669973792587046
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.69      0.42      0.52      1130
           1       0.77      0.74      0.76      2451

   micro avg       0.75      0.64      0.69      3581
   macro avg       0.73      0.58      0.64      3581
weighted avg       0.75      0.64      0.68      3581
 samples avg       0.34      0.32      0.32      3581



C:\Users\er_ja\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\er_ja\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


También podemos probar LinearSVC, en este caso el clasificador SVM que se usa kernel lineal:

In [51]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000))

param_grid = {
    'estimator__C': [0.1, 1.0],
    'estimator__tol': [1e-4, 1e-2],
    'estimator__max_iter': [1000, 10000, 50000],
    'estimator__intercept_scaling': [0.1, 1.0],
    'estimator__dual':[True, False],
}

# Paso 5: Configurar GridSearchCV
grid_search = GridSearchCV(multi_label_lsvm, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_split, y_train_split)

print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report

y_valid_pred2 = best_model.predict(X_valid_split)
print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred2))
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred2, zero_division=0 ))

C:\Users\er_ja\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Mejores parámetros encontrados:  {'estimator__C': 0.1, 'estimator__dual': False, 'estimator__intercept_scaling': 1.0, 'estimator__max_iter': 1000, 'estimator__tol': 0.01}
Accuracy en validación: 0.6710969674279296
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.70      0.41      0.52      1130
           1       0.78      0.74      0.76      2451

   micro avg       0.76      0.63      0.69      3581
   macro avg       0.74      0.57      0.64      3581
weighted avg       0.75      0.63      0.68      3581
 samples avg       0.34      0.32      0.32      3581



Comparamos los resultados con los que se obtienen a partir de OVO y OVR:

In [67]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

ovr = OneVsRestClassifier(LinearSVC(C=0.1, random_state=42, dual=False, max_iter=1000, tol=0.01)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
print("Reporte de clasificación Lineal OVR:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))


Accuracy en validación: 0.669973792587046
Reporte de clasificación Lineal OVR:
               precision    recall  f1-score   support

           0       0.69      0.42      0.52      1130
           1       0.77      0.74      0.76      2451

   micro avg       0.75      0.64      0.69      3581
   macro avg       0.73      0.58      0.64      3581
weighted avg       0.75      0.64      0.68      3581
 samples avg       0.34      0.32      0.32      3581



In [69]:
ovr = OneVsRestClassifier(SVC(C=1, random_state=42, gamma='scale', max_iter=50000, kernel='rbf')); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
print("Reporte de clasificación OVR:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))

Accuracy en validación: 0.669973792587046
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.69      0.42      0.52      1130
           1       0.77      0.74      0.76      2451

   micro avg       0.75      0.64      0.69      3581
   macro avg       0.73      0.58      0.64      3581
weighted avg       0.75      0.64      0.68      3581
 samples avg       0.34      0.32      0.32      3581



In [71]:
ovo = OneVsOneClassifier(LinearSVC(C=0.1, random_state=42, dual=False, max_iter=1000, tol=0.01)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
print("Reporte de clasificación Lineal OVR:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))

Accuracy en validación: 0.669973792587046
Reporte de clasificación Lineal OVR:
               precision    recall  f1-score   support

           0       0.69      0.42      0.52      1130
           1       0.77      0.74      0.76      2451

   micro avg       0.75      0.64      0.69      3581
   macro avg       0.73      0.58      0.64      3581
weighted avg       0.75      0.64      0.68      3581
 samples avg       0.34      0.32      0.32      3581



Parece claro que los métodos de OVR y OVO no están tan bien implementados como MultiOutputClassifier a la hora de tratar con problemas multietiqueta.